In [2]:
import os
import xlsxwriter
import numpy as np
from pylab import plot, grid, xlim, ylim, xlabel, ylabel, linspace, figure, legend
import pandas as pd

### Making spreadsheetof file names 

In [17]:
def FindDel(S):
    """This function takes a the first string in a list,(S) that starts with ENG, it finds the nomancature 
    the file uses for the begining of each file, e.g ENG44007 or ENGINX00305258, it shall also work if more numbers
    are added or the name changes so long as ENG followed by anything other than "." or "_" or " " is still 
    the start of each file"""
    for i in S:
        if i[0:3] == 'ENG':
            n = 1
            for j in i:
                if j == '.':
                    break
                elif j == ' ':
                    break
                elif j == '_':
                    break
                else: 
                    n = n+1 
            return n
        else:
            continue

directory = "D:\\"
array_names = []
file_years = []
long_array_names = []
for foldername in os.listdir(directory):
    if foldername[0:5] == 'cycle':
        n = FindDel(os.listdir("D:\\"+foldername))
        print(n)
        for filename in os.listdir("D:\\"+foldername):
            if filename[n:] in array_names: #stop repeated file names from entering 
                pass
            elif len(filename) < n+1: # Stop useless file names from entering (e.g copy)
                pass
            elif filename[n] == 's' or 'n':# Stop temporary files from entering (e.g .s01 or .n001)
                try:
                    int(filename[n+1]) # all temp files are 's' or 'n' followed by a int
                except: 
                    array_names.append(filename[n:])
                    long_array_names.append(filename)
                    file_years.append(foldername)
            else: # most unique file names go here 
                array_names.append(filename[n:])
                long_array_names.append(filename)
                file_years.append(foldername)
                
#print(file_years)
#print(array_names)
#print(long_array_names)

workbook   = xlsxwriter.Workbook('Filenames.xlsx')

worksheet1 = workbook.add_worksheet()
worksheet2 = workbook.add_worksheet()

worksheet1.write_column('A1', array_names)
worksheet2.write_column('A1', long_array_names)
worksheet1.write_column('B1', file_years)
worksheet2.write_column('B1', file_years)
workbook.close()

9
15
15


KeyboardInterrupt: 

### Extracting times

In [93]:
df = pd.read_csv("D:\\cycle_17_1\\ENGINX00269853_Lamp.txt", sep = "\t", names = ["Times", "other"])
li = df["Times"].values
print(li)

for i in range(len(li)):
    untdate = li[0].replace('T',' ')
    li = np.append(li,[untdate])
    li = np.delete(li,[0], axis = None)
        
print(li)  
d2f = pd.read_csv("D:\\cycle_04_1\\ENG44007_X.txt", sep = "\t", names = ["Times", "other"])
l2i = d2f["Times"].values
print(l2i)

#li = np.concatenate((li,l2i),axis=None)
#np.unique(li)
#print(li)

['2017-03-29T12:09:11' '2017-03-29T12:09:42' '2017-03-29T12:10:12' ...
 '2017-04-29T18:38:25' '2017-04-29T18:38:56' '2017-04-29T18:39:27']
['2017-03-29 12:09:11' '2017-03-29 12:09:42' '2017-03-29 12:10:12' ...
 '2017-04-29 18:38:25' '2017-04-29 18:38:56' '2017-04-29 18:39:27']
['2004-05-25 11:54:51' '2004-05-25 11:54:56' '2004-05-25 11:55:02' ...
 '2004-10-26 19:04:16' '2004-10-26 19:04:21' '2004-10-26 19:04:26']


In [89]:
a = np.array([1,2,3,3,4,1,13,23,213,57246,5472457,1])
b = np.array([13,23,1,5,3,7,6,96,8,6478,1,213])
#a = np.concatenate((a,b),axis=None)
#np.unique(a,axis=None)
#print(len(a))
a = np.append(a,[[1,3,4]])
#print(len(a))

c = np.array(["2017-03-29T12:09:11", "2017-03-29T12:09:42", "2017-03-29T12:10:12", "2017-03-29T12:10:42"
              "2017-03-29T12:11:12", "2017-03-29T12:11:42"])

print(c)
print(len(c))

for i in range(len(c)):
    untdate = c[0].replace('T',' ')
    c = np.append(c,[untdate])
    #print(li[0])
    c = np.delete(c,[0], axis = None)
    #print(c)
    
print(c)
print(len(c))

['2017-03-29T12:09:11' '2017-03-29T12:09:42' '2017-03-29T12:10:12'
 '2017-03-29T12:10:422017-03-29T12:11:12' '2017-03-29T12:11:42']
5
['2017-03-29 12:09:11' '2017-03-29 12:09:42' '2017-03-29 12:10:12'
 '2017-03-29 12:10:422017-03-29 12:11:12' '2017-03-29 12:11:42']
5


In [15]:
a = "kldgndfkhnopsmgnpksnhgo.txt"
b = "D:\\cycle_04_1\\ENG44007_Current.txt"
c = "2017-09-07T13:25:54"
d = "ENGINX00275375_cryo_temp2.txt"
print(c.replace('T', " "))
print(len(c))
print(d[0:14])

2017-09-07 13:25:54
19
ENGINX00275375


In [11]:
directory = "D:\\cycle_04_1"
a = np.array([])
for filename in os.listdir(directory):
    if filename[-4:] == ".txt":
        df = pd.read_csv(directory+"\\"+filename, sep = "\t", names = ["Times", "other"])
        li = df["Times"].values
        #print(filename)
        if os.stat(directory+"\\"+filename).st_size == 0:
            pass
        elif len(str(df.iloc[0].values[0])) < 18: # remove the files without time in the first colmn
                print(filename)
                pass
        elif li[0][10] =="T":
            for i in range(len(li)): # solve the T issue in some times 
                untdate = li[0].replace('T',' ')
                li = np.append(li,[untdate])
                li = np.delete(li,[0], axis = None)
            a = np.concatenate((a,li),axis=None)
        else: # Normal time columns
            a = np.concatenate((a,li),axis=None)
        #print(a)
print(len(a))
print(a)

                
        

ENG44007_dash_controls.txt
ENG44543_dash_controls.txt
ENG44641_dash_controls.txt
ENG44642_dash_controls.txt
ENG44839_dash_controls.txt
ENG44859_dash_controls.txt
ENG44864_dash_controls.txt
ENG44888_dash_controls.txt
ENG44904_dash_controls.txt
ENG44905_dash_controls.txt
ENG44907_dash_controls.txt
ENG44919_dash_controls.txt
ENG44961_dash_controls.txt
ENG44965_dash_controls.txt
ENG44999_dash_controls.txt
ENG45376_dash_controls.txt
ENG45672_dash_controls.txt
ENG45688_dash_controls.txt
ENG45775_dash_controls.txt
ENG45793_dash_controls.txt
ENG45918_dash_controls.txt
ENG45996_dash_controls.txt
ENG46024_dash_controls.txt
ENG46070_dash_controls.txt
ENG46076_dash_controls.txt
ENG46104_dash_controls.txt
ENG46140_dash_controls.txt
ENG46194_dash_controls.txt
ENG46368_dash_controls.txt
ENG46811_dash_controls.txt
ENG47176_dash_controls.txt
ENG47195_dash_controls.txt
ENG47285_dash_controls.txt
ENG47368_dash_controls.txt
ENG47394_dash_controls.txt
ENG47405_dash_controls.txt
ENG47508_dash_controls.txt
E

In [115]:
a = np.array([1])
b = np.array([1,2,3])
print(int(b))

TypeError: only size-1 arrays can be converted to Python scalars

In [109]:
a = np.array([1,2,3,4])
for i in a:
    a = np.concatenate((a,[1]), axis=None)
print(a)


[1 2 3 4 1 1 1 1]


In [18]:
def FindDel(S):
    """This function takes a the first string in a list,(S) that starts with ENG, it finds the nomancature 
    the file uses for the begining of each file, e.g ENG44007 or ENGINX00305258, it shall also work if more numbers
    are added or the name changes so long as ENG followed by anything other than "." or "_" or " " is still 
    the start of each file"""
    for i in S:
        if i[0:3] == 'ENG':
            n = 1
            for j in i:
                if j == '.':
                    break
                elif j == ' ':
                    break
                elif j == '_':
                    break
                else: 
                    n = n+1 
            return n
        else:
            continue

directory = "D:\\"
time = np.array([])
Experiment_index = np.array([])
for foldername in os.listdir(directory):
    if foldername[0:5] == 'cycle':
        n = FindDel(os.listdir("D:\\"+foldername))
        print(n)
        for filename in os.listdir("D:\\"+foldername):
            if filename[-4:] == ".txt":
                file_path = directory+foldername+"\\"+filename
                df = pd.read_csv(file_path, sep = "\t", names = ["Times", "other"])
                li = df["Times"].values
                if os.stat(file_path).st_size == 0: # remove empty files
                    pass
                elif len(str(df.iloc[0].values[0])) < 18: # remove the files without time in the first colmn
                    print(filename)
                    pass
                elif li[0][10] =="T":
                    for i in range(len(li)): # fix the T issue in some times 
                        untdate = li[0].replace('T',' ')
                        li = np.append(li,[untdate])
                        li = np.delete(li,[0], axis = None)
                    time = np.concatenate((a,li),axis=None)
                    Experiment_index = np.append(Experiment_index,[filename[:n-1]])
                else: # Normal time columns
                    time = np.concatenate((a,li),axis=None)
                    Experiment_index = np.append(Experiment_index,[filename[:n-1]])
                    #print(a)
print(len(time))
print(time)
print(Experiment_index)
print(len(Experiment_index))

9
ENG44007_dash_controls.txt


KeyboardInterrupt: 

In [127]:
df = pd.DataFrame({'col1': [1,2,3], 'col2': [4,5,6], 'col3': [4,5,6]})
# Extract second row (index: 1)
c = df.iloc[1].values
print(len(c))

3


### Making a dictionary from spreadsheet of filenames

In [ ]:
xls = pd.ExcelFile("C:\\Users\\zahaab\\ENGIN_X_DATA_CLEANING\\File_names_2013-19.xlsx")
df = xls.parse(sheet_name = 1)
df = df[["Filename", "What_is_inside"]]
true_name_translater = df.to_dict()#This has all unique file names as keys and the colomn name the data will go 
#as values
print(true_name_translater)

### Making functions for each file type 